## Import

In [ ]:
import matplotlib.pyplot as plt
from Core.Detection import Detection
from Core.Corner import Corner
from Core.interface import output, parseSingleInput, parseSingleInputHeterogeneous, Point
from Core.Connection import Connect
from Core.Utils import g_view_detail_hetero
from Core.GCN import GCNClassifier
from Core.ImageManager import Manager
# from Core.ocr import OCR
from Core.Utils import plot_result
import warnings

warnings.filterwarnings("ignore")

## Init Image

In [ ]:
# image_manager = Manager(r'C:\Users\PC\Desktop\public\images\058.png')
image_manager = Manager(r'C:\Users\PC\Desktop\Circuit-Dataset\images\1.png')

## Yolov8 Pose

In [ ]:
detector = Detection(model_path='./checkpoints/best-tuned.onnx')

In [ ]:
results = detector.predict(image=image_manager.image, conf=0.6, iou=0.5, imgsz=1024, draw=True)
line_width = detector.find_line_widths(results, image_manager.binary_image)
results = detector.fix(results, image_manager.binary_image, move2closest=True, find_pose=True, is_draw=True,
                       image=image_manager.image)
results, pose, mode = detector.results2custom(results, image_manager.binary_image)

In [ ]:
print(results)

In [ ]:
plt.imshow(image_manager.binary_image, cmap='gray')
plt.show()

## OCR

In [ ]:
# ocr_detector = OCR()
# ocr_result = ocr_detector.predict(image_manager, scale=0.8, conf=0.8, is_draw=True)
ocr_result = []

## HoughLines + DBSCAN

In [ ]:
print(line_width)
corner_detector = Corner(image_manager=image_manager)

alg = ['Harris', 'Shi-Tomasi', 'ORB', 'SIFT', 'BRISK']
# alg = ['Hough']
corner = corner_detector.predict(draw=True, threshold=100, results=results + ocr_result, line_width=line_width,
                                 algorithm=alg)
print(corner)

## Build Graph

In [ ]:
corner += pose
corner_detector.plot(corner)

In [ ]:
corner_detector.plot(pose)

In [ ]:
graph_builder = Connect(corner=corner, binary_image=image_manager.binary_image, results=results, mode=mode)

In [ ]:
# for i in range(len(corner)):
#     for j in range(i + 1, len(corner)):
#         judge = graph_builder.is_connected(corner[i], corner[j], threshold=line_width * 2)
#         if judge:
#             print(f'{i}-{j} {judge}')

In [ ]:
corner = graph_builder.build_graph(threshold=line_width * 2)
corner_detector.plot(corner)

## Output

In [ ]:
net = output(corner)
print(net)

## GCN/GCN Hetero

In [ ]:
is_hetero = False
if is_hetero:
    graph = parseSingleInputHeterogeneous(net)
    g_view_detail_hetero(graph.cpu())
    circuit_type = GCNClassifier('./checkpoints/GCNHetero.pt').val(graph)
else:
    graph = parseSingleInput(net)
    g_view_detail_hetero(graph.cpu())
    circuit_type = GCNClassifier('./checkpoints/GCN.pt').val(graph)
print(circuit_type)

## Full Output

In [ ]:
result = {
    "ckt_type": circuit_type,
    'ckt_netlist': net
}
# with open('output.txt', 'w') as f:
#     json.dump(result, f)
print(result)

## Plot All Result

In [ ]:
plot_result(corner, circuit_type, image_manager.image)